In [2]:
#------------ extract relevant conference in a file [paperid, conferenceid] -------------#
# !cat ../data/paperconferenceid.tsv | grep '1163450153\|1195049314\|1166315290\|2754292954\|1171345118' > ../data/HCI_paperids.tsv

cat: ../data/paperconferenceid.tsv: No such file or directory


In [1]:
#------------ paper year -------------# data source:/data/pcsdata/selected_MAG_metadata/paperyear.tsv
from itertools import chain
import pandas as pd
# data_HCI = list(chain(*pd.read_csv('../data/HCI_paperids.tsv',sep=',').values.tolist()))
data_HCI = pd.read_csv('HCI_paperids.tsv',sep='\t')
data_HCI.head(10)
conf_name_map = {1163450153:'CHI', 1195049314:'CSCW', 1171345118:'UBI', 1166315290:'UIST'}

In [4]:
data_HCI.to_csv('temp.csv',encoding='utf-8')

In [2]:
results = {'result_CHI':[], 'result_UBI':[], 'result_CSCW':[], 'result_UIST':[]}
ori_file = open('../data/paperyear.tsv')
conf_name_map = {1163450153:'CHI', 1195049314:'CSCW', 1171345118:'UBI', 1166315290:'UIST'}
# conf_name_map could also load by category
# data_category = pd.read_csv('../data/category.tsv',sep=',')
# data_category = data_category.loc[data_category['category']=="AI"]
# for row in data_category.iterrows():
#     conf_name_map[row[0]] = row[1]

iter_f = iter(ori_file)
first_line = True
for row in iter_f:
    if first_line:
        first_line = False
        continue
    paperid_str = str(row).split()[0] # paperid
    paperrow_str = str(row).split() # paper row
    paperid = int(paperid_str)
    if paperid in data_HCI['paperid'].values:
        results['result_{}'.format(conf_name_map[int(data_HCI.loc[data_HCI['paperid']==paperid]['confid'].values)])].append(list(paperrow_str))
for name, result in results.items():
    result_pd = pd.DataFrame(data=result, columns=['paperid', 'year'])
    result_pd.to_csv('paperyear_' + str(name) + '.tsv')
print(">>> done")

FileNotFoundError: [Errno 2] No such file or directory: '../data/paperyear.tsv'

In [ ]:
#------------ paper titles -------------# data source: /data4/pcs/papertitle.tsv
import os
from itertools import chain
import pandas as pd

results_title = {'result_CHI':[], 'result_UBI':[], 'result_CSCW':[], 'result_UIST':[]}
ori_pd = pd.read_csv('../data/papertitle.tsv', skiprows=1)
pd.set_option("display.max_colwidth", 100000)

for row in ori_pd.iterrows():
    paperid_str = str(row[1]).split('\\t')[1].split()[1] # paperid
    paper_title = str(row[1]).split('\\t')[2].split('\n')[0]
    paperrow_str = [paperid_str, paper_title]
    paperid = int(paperid_str)
    if paperid in data_HCI['paperid'].values:
        results['result_{}'.format(conf_name_map[int(data_HCI.loc[data_HCI['paperid']==paperid]['confid'].values)])].append(list(paperrow_str))
        
for name, result in results_title.items():
    result_pd = pd.DataFrame(data=result, columns=['paperid', 'title'])
    result_pd.to_csv('papertitle_' + str(name) + '.tsv')
print(">>> done")

In [ ]:
#------------ paper author order -------------#
from itertools import chain
import pandas as pd

results_author = {'result_CHI':[], 'result_UBI':[], 'result_CSCW':[], 'result_UIST':[]}
ori_paperauthor = pd.read_csv('../data/paperauthororder.tsv',skiprows=1)
for row in ori_paperauthor.iterrows():
    paperid_str = str(row[1]).split('    ')[1].split('\\t')[0] # paperid
    paperrow_str = str(row[1]).split('    ')[1].split('\n')[0].split('\\t') # paper row
    paperid = int(paperid_str)

    if paperid in data_HCI['paperid'].values:
        results_author['result_{}'.format(conf_name_map[int(data_HCI.loc[data_HCI['paperid']==paperid]['confid'].values)])].append(list(paperrow_str))

for name, result in results_author.items():
    result_pd = pd.DataFrame(data=result, columns=['paperid', 'authorid', 'authororder'])
    result_pd.to_csv('paperauthororder_' + str(name) + '.tsv')
print(">>> done")

In [9]:
## faster implementation for detecting author affiliation at paper level
f = open('paperauthoridaffiliationname.tsv')
paper_author_affiliation = {}
paper_author_id = {}
line = f.readline()
while line:
    paperid = line.split('\t')[0]
    authorid = line.split('\t')[1]
    affiliation = line.split('\t')[2][:-1]
    if paperid not in paper_author_affiliation:
        paper_author_id[paperid] = [authorid]
        paper_author_affiliation[paperid] = [affiliation]
    else:
        paper_author_id[paperid].append(authorid)
        paper_author_affiliation[paperid].append(affiliation)
    line = f.readline()

In [27]:
## get author affiliation, id level information related to hci conferences
Paperid = []
Affiliation = []
AuthorId = []
for paperid in data_HCI['paperid']:
    paperid = str(paperid)
    try:
        
        Affiliation.append(paper_author_affiliation[paperid])
        AuthorId.append(paper_author_id[paperid])
        Paperid.append(paperid)
    except:
        pass
df = pd.DataFrame({'paperid':Paperid, 'authorid':AuthorId,'Affiliation':Affiliation})
df.to_csv('HCI_paper_authorid_affiliation.tsv',sep='\t', index=False,encoding='utf-8')

In [2]:
#------------ paper affiliations -------------#
import os
from itertools import chain
import pandas as pd

results_affiliation = {'result_CHI':[], 'result_UBI':[], 'result_CSCW':[], 'result_UIST':[]}
#ori_pd = pd.read_csv('../data/paperauthoridaffiliationname.tsv', sep='\t', skiprows=1)
ori_pd = pd.read_csv('/dfs/scratch0/hanchcao/Transfer_CS/transfer_cs/108362-V14/selected-MAG-metadata-2018/paperauthoridaffiliationname.tsv', sep='\t', skiprows=1)
print ('load data complete!')
pd.set_option("display.max_colwidth", 100000)

count = 0
for row in ori_pd.iterrows():
    count += 1
    if count%10000 ==0 :
        print (count)
    paperid_str = str(row[1]).split('\n')[0].split()[1]
    authorid = str(row[1]).split('\n')[1].split()[1]
    affiliationname = str(row[1]).split('\n')[2].split()[1]
    paperrow_str = [paperid_str, authorid, affiliationname]
    paperid = int(paperid_str)

    if paperid in data_HCI['paperid'].values:
        results_affiliation['result_{}'.format(conf_name_map[int(data_HCI.loc[data_HCI['paperid']==paperid]['confid'].values)])].append(list(paperrow_str))

for name, result in results_affiliation.items():
    result_pd = pd.DataFrame(data=result, columns=['paperid', 'authorid', 'affiliationname'])
    result_pd.to_csv('authoraffname_' + str(name) + '.tsv', encoding='utf-8')
print(">>> done")

load data complete!


NameError: name 'conf_name_map' is not defined

In [4]:
count = 0
for row in ori_pd.iterrows():
    count += 1
    if count%10000 ==0 :
        print (count)
    paperid_str = str(row[1]).split('\n')[0].split()[1]
    authorid = str(row[1]).split('\n')[1].split()[1]
    affiliationname = str(row[1]).split('\n')[2].split()[1]
    paperrow_str = [paperid_str, authorid, affiliationname]
    paperid = int(paperid_str)

    if paperid in data_HCI['paperid'].values:
        results_affiliation['result_{}'.format(conf_name_map[int(data_HCI.loc[data_HCI['paperid']==paperid]['confid'].values)])].append(list(paperrow_str))

for name, result in results_affiliation.items():
    result_pd = pd.DataFrame(data=result, columns=['paperid', 'authorid', 'affiliationname'])
    result_pd.to_csv('authoraffname_' + str(name) + '.tsv')
print(">>> done")

10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000


KeyboardInterrupt: 

In [ ]:
#------------ paper citations -------------# data source: /data4/pcs/papercitations
import os
from itertools import chain
import pandas as pd

results_citing = {'result_CHI':[], 'result_UBI':[], 'result_CSCW':[], 'result_UIST':[]}
results_cited = {'result_CHI':[], 'result_UBI':[], 'result_CSCW':[], 'result_UIST':[]}

# all citation raw file are listed in papercitations directory
files = os.listdir("../data/papercitations")
for file in files:
    firstline = True
    paper_citation_file = open(os.path.join("../data/papercitations", file))
    iter_f = iter(paper_citation_file)
    for row in iter_f:
        if firstline:
            firstline = False
            continue
        if len(str(row).split()) < 2:
            continue
        citing_paperid_str = str(row).split()[0] # citingpaperid
        cited_paperid_str = str(row).split()[1]
        paperrow_str = [citing_paperid_str, cited_paperid_str]
        paperid = int(citing_paperid_str)
        paperid2 = int(cited_paperid_str)

        if paperid in data_HCI['paperid'].values:
            results_citing['result_{}'.format(conf_name_map[int(data_HCI.loc[data_HCI['paperid']==paperid]['confid'].values)])].append(list(paperrow_str))

        if paperid2 in data_HCI['paperid'].values:
            results_cited['result_{}'.format(conf_name_map[int(data_HCI.loc[data_HCI['paperid']==paperid2]['confid'].values)])].append(list(paperrow_str))

for name, result in results_citing.items():
    result_pd = pd.DataFrame(data=result, columns=['citingpaperid', 'citedpaperid'])
    result_pd.to_csv('paperciting_' + str(name) + '.tsv')

for name, result in results_cited.items():
    result_pd = pd.DataFrame(data=result, columns=['citingpaperid', 'citedpaperid'])
    result_pd.to_csv('papercited_' + str(name) + '.tsv')

print(">>> done")

In [ ]:
#------------ paper citations science -------------# data source: /data/pcsdata/pcs.tsv
from itertools import chain
import pandas as pd

results_pcs = {'result_CHI':[], 'result_UBI':[], 'result_CSCW':[], 'result_UIST':[]}
ori_file = open('../data/pcs.tsv')
iter_f = iter(ori_file)
first_line = True
for row in iter_f:
    if first_line:
        first_line = False
        continue
    paperrow_str = str(row).split() # paper row
    paperid_str = paperrow_str[2]
    paperid = int(paperid_str)
    if paperid in data_HCI['paperid'].values:
        results_pcs['result_{}'.format(conf_name_map[int(data_HCI.loc[data_HCI['paperid']==paperid]['confid'].values)])].append(list(paperrow_str))

for name, result in results_pcs.items():
    result_pd = pd.DataFrame(data=result, columns=['reftype', 'confscore', 'paperid', 'patent'])
    result_pd.to_csv('papercitationscience_' + str(name) + '.tsv')

In [40]:
#------------ patent inventors -------------# data source:pcs/patent_dblpdata/patent_inventor.tsv
from itertools import chain
import pandas as pd
patent_data_HCI = pd.read_csv('../data/papercitationscience_result.tsv',sep='\t')
patent_data_HCI = patent_data_HCI.merge(data_HCI, left_index=True, right_index=True)
patent_data_HCI.head(5)

,",reftype,confscore,paperid,patent",paperid,confid
0,"0,app,9,2134161165,10001804",2243662716,1163450153
1,"1,app,9,1967451823,10001885",2244265604,1163450153
2,"2,app,9,1967451823,10001888",2244428935,1163450153
3,"3,app,10,2005198142,10001888",2244483578,1163450153
4,"4,app,10,2152754623,10001897",2244508988,1163450153


In [ ]:
results_inventors = {'result_CHI':[], 'result_UBI':[], 'result_CSCW':[], 'result_UIST':[]}

ori_patentinventors = pd.read_csv('../data/patent_inventor.tsv',skiprows=1)
for row in ori_patentinventors.iterrows():
    patent_id = str(row[1]).split('    ')[1].split('\\t')[0]
    patentinventors_row = str(row[1]).replace("\"","").split('    ')[1].split('\n')[0].split('\\t')

    if paperid in data_HCI['paperid'].values:
        results_inventors['result_{}'.format(conf_name_map[int(patent_data_HCI.loc[patent_data_HCI['paperid']==paperid]['confid'].values)])].append(list(patentinventors_row))
        
for name, result in results_inventors.items():
    result_pd = pd.DataFrame(data=result, columns=['patent_id', 'inventor_id', 'location_id'])
    result_pd.to_csv('patentinventors_' + str(name) + '.tsv')
print(">>> done")

In [ ]:
#------------ patent abstract -------------# data source: /data4/pcs/abstract/brf_sum_text.tsv
from itertools import chain
import pandas as pd
results_abstract = {'result_CHI':[], 'result_UBI':[], 'result_CSCW':[], 'result_UIST':[]}

ori_patent_abstract = pd.read_csv('../data/brf_sum_text.tsv', sep='\t', skiprows=1)
pd.set_option("display.max_colwidth", 100000)
for row in ori_patent_abstract.iterrows():
    uuid = str(row[1]).split('\n')[0].split()[1].strip("\n")
    patent_id = str(row[1]).split('\n')[1].split()[1].strip("\n")
    abstract = str(row[1]).split('\n')[2].split('         ')[1]
    patent_abstract_row = [uuid, patent_id, abstract]
    
    if paperid in data_HCI['paperid'].values:
        results_abstract['result_{}'.format(conf_name_map[int(patent_data_HCI.loc[patent_data_HCI['paperid']==paperid]['confid'].values)])].append(list(patent_abstract_row))

for name, result in results_inventors.items():
    result_pd = pd.DataFrame(data=result, columns=['uuid', 'patent_id', 'abstract'])
    result_pd.to_csv('patent_abstract_' + str(name) + '.tsv')
print(">>> done")

In [ ]:
#------------ patent year -------------# data source:/patentsview/patent.tsv
from itertools import chain
import pandas as pd
import os

HCI_ori_pd = None
for path,directory,files in os.walk('../data/patentinventors/'):
    # here, we have conf_id name of CHI, CSCW, UBI, UIST
    for file in files:
        ori_pd = pd.read_csv(os.path.join(path, file))
        conf_id = file[file.index('_') + 1:-4]
        # add conf_id property to patent data
        ori_pd['conf_id'] = conf_id
        if HCI_ori_pd is None:
            HCI_ori_pd = ori_pd
        else:
            HCI_ori_pd = pd.concat([HCI_ori_pd, ori_pd],axis=0)
# extract patent year data
print("start load")
patent_pd = pd.read_csv('patent.tsv',sep='\t')
# merge ori_conf_pd_map
print("start merge")
HCI_ori_pd = HCI_ori_pd.merge(patent_pd, left_on='patent_id', right_on='id')
HCI_ori_pd.to_csv('patent_year_inventor.tsv')
HCI_ori_pd.head(5)